연금 전환시

연금수령한도
$\frac{연금계좌 평가액}{11 - 연금수령 연차} \times 120\%$

퇴직연금은 수령한도 내에서 찾을시에 30% 감면

[연금인출순서](https://www.pwc.com/kr/ko/insights/issue-brief/retirement-pension4.png)


ㅂ연금 = 국민연금 + 퇴직연금(IRP) + 개인연금

### 세액공제
![](https://mybiz.pay.naver.com/api/files/202402/8-2-1926172304.png)
### 과세이연
- 국민연금 : 납부시 전액 소득공제
- 연금계좌에서 발생한 소득에 대해 즉시 세금을 부과하지 않고, 연금 개시 후 연금액에 따라 세금 부과
### 저율과세
- 국민연금 : 원천징수 (종합과세)
- 퇴직연금 : 퇴직소득세의 70% ~ 60%
- 개인연금 : 5.5% - 3.3% 저율과세


In [5]:
import java.text.NumberFormat
import java.util.Locale

fun 공제율(급여 : Int): Double {
    return if (급여 <= 5500_0000) 0.15 else 0.12
}

val 납입액 = 900_0000
val 급여 = 1_0000_0000

val 세액공제 = 납입액 * 공제율(급여)

val formatter = NumberFormat.getCurrencyInstance(Locale.KOREA)

println("연금 납입액: ${formatter.format(납입액)}")
println("세액공제: ${formatter.format(세액공제)}")


연금 납입액: ₩9,000,000
세액공제: ₩1,080,000
